In [1]:
import numpy as np 
import json

In [2]:
def map_feature(x1, x2):
    degree = 6
    out = np.ones([len(x1), (degree + 1) * (degree + 2) // 2])
    idx = 1

    for i in range(1, degree + 1):
        for j in range(0, i + 1):
            a1 = x1 ** (i - j)
            a2 = x2 ** j
            out[:, idx] = a1 * a2
            idx += 1

    return out

In [3]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_cost(theta, X, y, lambd):
    m = len(y)
    h = sigmoid(X @ theta)
    cost = (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    reg_term = (lambd / (2 * m)) * np.sum(theta[1:]**2)  # Exclude theta_0 from regularization
    return cost + reg_term

def compute_gradient(theta, X, y, lambd):
    m = len(y)
    h = sigmoid(X @ theta)
    grad = (X.T @ (h - y)) / m
    grad[1:] += (lambd / m) * theta[1:]  # Regularization for j >= 1
    return grad

def gradient_descent(X, y, theta, alpha, lambd, num_iter):
    m = len(y)
    costs = []

    for _ in range(num_iter):
        cost = compute_cost(theta, X, y, lambd)
        grad = compute_gradient(theta, X, y, lambd)
        theta -= alpha * grad
        costs.append(cost)

    return theta, costs

In [4]:
def predict(theta, X):
    return (sigmoid(X @ theta) >= 0.5).astype(int)

In [5]:
def evaluate(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    tn = np.sum((y_true == 0) & (y_pred == 0))

    accuracy = np.mean(y_true == y_pred)
    precision_pos = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall_pos = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score_pos = 2 * (precision_pos * recall_pos) / (precision_pos + recall_pos) if (precision_pos + recall_pos) > 0 else 0

    precision_neg = tn / (tn + fn) if (tn + fn) > 0 else 0
    recall_neg = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1_score_neg = 2 * (precision_neg * recall_neg) / (precision_neg + recall_neg) if (precision_neg + recall_neg) > 0 else 0

    return {
        "Accuracy": accuracy * 100,
        "Precision_Positive": precision_pos,
        "Recall_Positive": recall_pos,
        "F1-score_Positive": f1_score_pos,
        "Precision_Negative": precision_neg,
        "Recall_Negative": recall_neg,
        "F1-score_Negative": f1_score_neg
    }

In [7]:
def main():
    with open('config.json') as config_file:
        config = json.load(config_file)

    alpha = config["Alpha"]
    lambd = config["Lambda"]
    num_iter = config["NumIter"]

    # Read the training data from training_data.txt
    data = np.loadtxt('training_data.txt', delimiter=',')
    X_raw = data[:, :-1]
    y = data[:, -1]

    X = map_feature(X_raw[:, 0], X_raw[:, 1])
    n = X.shape[1]  # Number of features after mapping
    theta = np.zeros(n)

    # Train the logistic regression model using gradient descent
    theta, costs = gradient_descent(X, y, theta, alpha, lambd, num_iter)
    model = {
        "Theta": theta.tolist(),
        "Alpha": alpha,
        "Lambda": lambd
    }

    with open('model.json', 'w') as model_file:
        json.dump(model, model_file)

    # Evaluate the model on the training dataset and save the results
    y_pred = predict(theta, X)
    evaluation_result = evaluate(y, y_pred)

    with open('classification_report.json', 'w') as report_file:
        json.dump(evaluation_result, report_file)

In [ ]:
main()